In [1]:
import xarray as xr
import os
from cae_tools.models.conv_ae_model import ConvAEModel
from cae_tools.models.model_evaluator import ModelEvaluator

C:\Users\matth\anaconda3\envs\reading\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [14]:
## LOAD ORIGINAL DATA AND RE-FORMAT TO SEPARATE TRAIN AND TEST FILES
data_path = r"C:\Users\matth\Desktop\data" # path to original data file
os.chdir(data_path)
ds = xr.open_dataset('l3_blended_l4_extracts_gR_201701.nc')
ds = xr.Dataset({
    "sstL4": ds.sstL4,
    "gRsst": ds.gRsst}) # isolate only sstL4 and gRsst
train_percent = 0.85
split_index = int(train_percent * len(ds.i))

train_ds = ds.isel(i=slice(0, split_index))
test_ds= ds.isel(i=slice(split_index, None))

channel = [0]
train_ds = train_ds.expand_dims(dim='channel', axis = 1)
test_ds = test_ds.expand_dims(dim='channel', axis = 1)

# train_ds_temp = train_ds
# train_ds_temp['channel'] = train_ds.i
# train_ds_temp['i'] = train_ds.channel

train_ds.to_netcdf('train_ds.nc')
test_ds.to_netcdf('test_ds.nc')

train_path = "train_ds.nc"
test_path = "test_ds.nc"

train_ds

<xarray.Dataset>
Dimensions:  (i: 5154, y: 48, x: 48, channel: 1)
Coordinates:
    lat      (i, y) float64 -50.97 -50.92 -50.88 -50.83 ... -38.72 -38.67 -38.62
    lon      (i, x) float64 20.02 20.08 20.12 20.17 ... 161.2 161.3 161.3 161.4
Dimensions without coordinates: i, y, x, channel
Data variables:
    sstL4    (i, channel, y, x) float32 275.6 275.7 275.8 ... 291.5 291.4 291.4
    gRsst    (i, channel, y, x) float32 275.6 275.7 275.7 ... 291.5 291.4 291.4

In [15]:
# RELOAD DATA
train_ds = xr.open_dataset(train_path)
test_ds = xr.open_dataset(test_path)

In [17]:
mt = ConvAEModel(fc_size=8, encoded_dim_size=4, nr_epochs=500) 
mt.train(["sstL4"], "gRsst", training_ds=train_ds, testing_ds=test_ds, training_paths=train_path, testing_paths=test_path)
print(mt.summary())

Running on device: cpu
    0 0.038711 0.019593
   10 0.006935 0.003034
   20 0.005423 0.002431
   30 0.004789 0.002056
   40 0.004338 0.001709
   50 0.004178 0.001590


KeyboardInterrupt: 